In [2]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
# Loading data
train = pd.read_csv('../../data/class_regression/prepocessed_train_data.csv', index_col='id')
test = pd.read_csv('../../data/class_regression/prepocessed_test_data.csv', index_col='id')
X_train = train.drop(columns=['playtime_forever', 'is_play'])
y1_train = train['playtime_forever']
y2_train = train['is_play']

In [18]:
xgbc = XGBClassifier()
xgbc.fit(X_train, y2_train)
rfr = RandomForestRegressor()
rfr.fit(X_train, y1_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [4]:
cross_val_score(xgbc, X_train, y2_train, cv=10).mean()

0.7164052287581699

In [19]:
cross_val_score(rfr, X_train, y1_train, cv=10).mean()

-0.3868279404295259

In [20]:
is_play = xgbc.predict(test)
playtime_forever = rfr.predict(test)
submission = pd.DataFrame(data={
    'playtime_forever': playtime_forever,
    'is_play': is_play
})
submission.loc[submission['is_play'] == 0, 'playtime_forever'] = 0
submission = submission.drop(columns=['is_play'])
submission.index.name = 'id'
submission.to_csv('../../submission/class_regression/xgbc_rfr.csv')